In [1]:
#Required packages
import pandas as pd
from fitparse import FitFile
from sdv.single_table import GaussianCopulaSynthesizer
from sdv.metadata import Metadata
import numpy as np

In [28]:
#File contains heart rate, stress level and respiration data
f=FitFile("407432282869_WELLNESS.fit")
data_f=[]

#Checks the existing message names which can be filtered on
message_names = set()
for msg in g.get_messages():
    message_names.add(msg.name)
for name in sorted(message_names):
    print(name)

#Only select rows with heart rate data
for record in f.get_messages("monitoring"):
    record_data={}
    for field in record:
        record_data[field.name]=field.value
    data_f.append(record_data)

#Make into pandas dataframe
df_f=pd.DataFrame(data_f)

#metadata is extracted and used to make a synthesizer
metadata = Metadata.detect_from_dataframe(df_f) 
synthesizer = GaussianCopulaSynthesizer(metadata) 
synthesizer.fit(df_f) #synthesizer is fitted onto the data
hr_synthetic = synthesizer.sample(num_rows=400) #synthesizer generates rows
hr_synthetic = hr_synthetic.drop(["unknown_37", "unknown_38", "activity_type", "current_activity_type_intensity","intensity"], axis=1)

n = len(hr_synthetic)
hours = 24

hr_synthetic["hour_bin"] = (np.arange(n) * hours // n)  #integer hour index
#create timestamps for each hour
start = "00:00:00"
hour_timestamps = pd.date_range(start=start, periods=hours, freq="h")
#aggregate each hour_bin
hr_hourly = hr_synthetic.groupby("hour_bin").mean(numeric_only=True)
hr_hourly["heart_rate"] = round(hr_hourly["heart_rate"])
hr_hourly["hour_timestamp"] = hour_timestamps

#Convert to csv
hr_hourly.to_csv("heart_rate_hourly.csv",index=False)

C:\Users\Aleksandra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sdv\single_table\base.py:134: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


In [27]:
#Read the .fit file
g=FitFile("407432248700_HRV_STATUS.fit")
data_g=[]

#Checks the existing message names which can be filtered on
message_names = set()
for msg in g.get_messages():
    message_names.add(msg.name)
for name in sorted(message_names):
    print(name)

#Only select rows with heart rate variability data
for record in g.get_messages("unknown_371"):
    record_data={}
    for field in record:
        record_data[field.name]=field.value
    data_g.append(record_data)

#Make into pandas dataframe
df_g=pd.DataFrame(data_g)

#metadata is extracted and used to make a synthesizer
metadata = Metadata.detect_from_dataframe(df_g) 
synthesizer = GaussianCopulaSynthesizer(metadata) 
synthesizer.fit(df_g) #synthesizer is fitted onto the data
hrv_synthetic = synthesizer.sample(num_rows=400) #synthesizer generates rows

hrv_synthetic = hrv_synthetic.rename(columns={"unknown_0":"hrv_value", "unknown_253":"timestamp_16"})

#create timestamps for each hour
n = len(hrv_synthetic)
hours = 24
hrv_synthetic["hour_bin"] = (np.arange(n) * hours // n) 
start = "00:00:00"
hour_timestamps = pd.date_range(start=start, periods=hours, freq="h")
#aggregate each hour_bin
hrv_hourly = hrv_synthetic.groupby("hour_bin").mean(numeric_only=True)
hrv_hourly["hrv_value"] = round(hrv_hourly["hrv_value"])
hrv_hourly["hour_timestamp"] = hour_timestamps

#Convert to csv
hrv_hourly.to_csv("heart_rate_variability_hourly.csv",index=False)

device_info
file_creator
file_id
timestamp_correlation
unknown_22
unknown_370
unknown_371


C:\Users\Aleksandra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sdv\single_table\base.py:134: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(
